# Market data - Technical Indicators

### Import

In [6]:
import numpy as np
import pandas as pd
from pathlib import Path 
import datatable as dt

from ta.volatility import BollingerBands
from ta.trend import EMAIndicator
from ta.momentum import AwesomeOscillatorIndicator

### Get data

In [2]:
filepath = Path('../../data/csv/crypto_market_data.csv')

In [3]:
df = dt.fread(filepath).to_pandas() 
df = df.T.set_index([0, 1]).T
df = df.set_index('')
df = df.astype(float)
df.index.names = ['date']
df.head(3)

0                          USDT-USD                                          \
1                         Adj Close     Close      High       Low      Open   
date                                                                          
2020-04-07 00:00:00+00:00  0.998488  0.998488  1.000555  0.990544  0.993490   
2020-04-07 01:00:00+00:00  0.999708  0.999708  1.000824  0.994457  0.998814   
2020-04-07 02:00:00+00:00  1.000362  1.000362  1.004532  0.997544  1.000190   

0                                            BTC-USD               \
1                                Volume    Adj Close        Close   
date                                                                
2020-04-07 00:00:00+00:00  0.000000e+00  7366.915527  7366.915527   
2020-04-07 01:00:00+00:00  6.818161e+08  7355.732910  7355.732910   
2020-04-07 02:00:00+00:00  1.158300e+09  7311.450195  7311.450195   

0                                                    ...   ADA-USD            \
1                                 High          Low  ...      High       Low   
date                                                 ...                       
2020-04-07 00:00:00+00:00  7407.448730  7268.242188  ...  0.036678  0.035576   
2020-04-07 01:00:00+00:00  7383.069824  7350.611816  ...  0.036497  0.036083   
2020-04-07 02:00:00+00:00  7355.026367  7290.237793  ...  0.036182  0.035433   

0                                                BNT-USD                      \
1                              Open     Volume Adj Close     Close      High   
date                                                                           
2020-04-07 00:00:00+00:00  0.035681        0.0  0.198634  0.198634  0.200798   
2020-04-07 01:00:00+00:00  0.036118  1867280.0  0.197310  0.197310  0.198583   
2020-04-07 02:00:00+00:00  0.036182  2287464.0  0.195936  0.195936  0.197029   

0                                                       
1                               Low      Open   Volume  
date                                                    
2020-04-07 00:00:00+00:00  0.196712  0.196712      0.0  
2020-04-07 01:00:00+00:00  0.197102  0.198583   5230.0  
2020-04-07 02:00:00+00:00  0.195246  0.196872  49303.0  

[3 rows x 60 columns]

In [4]:
df_btc = df['BTC-USD']

### Compute technical indicators

In [15]:
df_btc_ta = pd.DataFrame([])
df_btc_ta['Close'] = df_btc.Close

In [17]:
# Bollinger band
BB = BollingerBands(close=df_btc.Close, window=20, window_dev=2)
df_btc_ta['BB_high'] = BB.bollinger_hband() 
df_btc_ta['BB_low'] = BB.bollinger_lband()    
df_btc_ta['BB_width'] = BB.bollinger_wband() 

# Moving average
SMA_BBW = AwesomeOscillatorIndicator(high=df_btc_ta.BB_width, low=df_btc_ta.BB_width, window1=10, window2=50)
SMA_Vol = AwesomeOscillatorIndicator(high=df_btc.Volume, low=df_btc.Volume, window1=10, window2=50)
df_btc_ta['SMA_BBW'] = SMA_BBW.awesome_oscillator()
df_btc_ta['SMA_Vol'] = SMA_Vol.awesome_oscillator() 

# Exponential moving average
EMA_fast = EMAIndicator(close=df_btc.Close, window=12)
EMA_slow = EMAIndicator(close=df_btc.Close, window=26)
df_btc_ta['EMA_fast'] = EMA_fast.ema_indicator() 
df_btc_ta['EMA_slow'] = EMA_slow.ema_indicator()
df_btc_ta['MACD'] = df_btc_ta['EMA_slow'] - df_btc_ta['EMA_fast']

# MACD
EMA_MACD = EMAIndicator(close=df_btc_ta.MACD, window=9)
df_btc_ta['EMA_MACD'] = EMA_MACD.ema_indicator() 

# Highest High & Lowest Low
df_btc_ta['HH'] = df_btc.High.rolling(20).max()
df_btc_ta['LL'] = df_btc.Low.rolling(20).min()

In [20]:
df_btc_ta = df_btc_ta.dropna()
df_btc_ta.head()

,Close,BB_high,BB_low,BB_width,SMA_BBW,SMA_Vol,EMA_fast,EMA_slow,MACD,EMA_MACD,HH,LL
date,,,,,,,,,,,,
2020-04-09 20:00:00+00:00,7254.513672,7329.607505,7251.915152,1.065627,-1.480833,2.109254e+08,7280.083462,7287.614550,7.531088,3.132471,7337.893066,7179.094238
2020-04-09 21:00:00+00:00,7273.934570,7327.833311,7250.361709,1.062842,-1.457221,2.290828e+08,7279.137478,7286.601218,7.463740,3.998724,7327.550781,7179.094238
2020-04-09 22:00:00+00:00,7275.651367,7325.640758,7249.332240,1.047117,-1.424009,4.009165e+07,7278.601154,7285.790118,7.188964,4.636772,7327.550781,7179.094238
2020-04-10 00:00:00+00:00,7299.950195,7326.467842,7249.426250,1.057110,-1.377465,-1.624236e+07,7281.885621,7286.839013,4.953391,4.700096,7327.550781,7179.094238
2020-04-10 01:00:00+00:00,7283.327637,7325.562422,7248.925419,1.051660,-1.320730,-2.388591e+07,7282.107470,7286.578911,4.471441,4.654365,7327.550781,7179.094238


### Save features

In [21]:
filepath = Path('../csv/features_btc_ta.csv')

In [22]:
df_btc_ta.to_csv(filepath)